Задание 19. Связь между образом жизни, питанием и состоянием здоровья населения.

In [65]:
import pandas as pd
import kagglehub

# Скачиваем датабазу
path = kagglehub.dataset_download("atharvasoundankar/mental-health-and-lifestyle-habits-2019-2024")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'mental-health-and-lifestyle-habits-2019-2024' dataset.
Path to dataset files: /kaggle/input/mental-health-and-lifestyle-habits-2019-2024


In [66]:
# Читаем датабазу и выводим её
df = pd.read_csv(path + '/Mental_Health_Lifestyle_Dataset.csv')
df.head()

,Country,Age,Gender,Exercise Level,Diet Type,Sleep Hours,Stress Level,Mental Health Condition,Work Hours per Week,Screen Time per Day (Hours),Social Interaction Score,Happiness Score
0,Brazil,48,Male,Low,Vegetarian,6.3,Low,NaN,21,4.0,7.8,6.5
1,Australia,31,Male,Moderate,Vegan,4.9,Low,PTSD,48,5.2,8.2,6.8
2,Japan,37,Female,Low,Vegetarian,7.2,High,NaN,43,4.7,9.6,9.7
3,Brazil,35,Male,Low,Vegan,7.2,Low,Depression,43,2.2,8.2,6.6
4,Germany,46,Male,Low,Balanced,7.3,Low,Anxiety,35,3.6,4.7,4.4


Чтобы предсказать здоровье населения "Mental Health Condition" (упростим до есть или нет заболевания) мы будем использовать столбцы: "Exercise Level" и "Diet Type"

In [67]:
# Создадим копию датафрейма для работы
df_ml = df[['Exercise Level', 'Diet Type', 'Mental Health Condition']].copy()

# Преобразуем целевую переменную в бинарную (есть/нет заболевания)
df_ml['Target'] = df['Mental Health Condition'].apply(
    lambda x: 1 if x == ('Depression' or 'Bipolar' or 'Anxiety' or 'PTSD') else 0)

# Преобразуем категориальные признаки в числовые (One-Hot Encoding)
df_ml = pd.get_dummies(df_ml, columns=['Exercise Level', 'Diet Type'], drop_first=True)

print("Размер датасета после обработки:", df_ml.shape)
print("\nПервые 5 строк:")
df_ml.head()

Размер датасета после обработки: (3000, 8)

Первые 5 строк:


,Mental Health Condition,Target,Exercise Level_Low,Exercise Level_Moderate,Diet Type_Junk Food,Diet Type_Keto,Diet Type_Vegan,Diet Type_Vegetarian
0,NaN,0,True,False,False,False,False,True
1,PTSD,0,False,True,False,False,True,False
2,NaN,0,True,False,False,False,False,True
3,Depression,1,True,False,False,False,True,False
4,Anxiety,0,True,False,False,False,False,False


In [68]:
from sklearn.model_selection import train_test_split

x = df_ml.drop('Target', axis=1)
y = df_ml['Target']

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y)

print(f"Тренировочные данные: {x_train.shape[0]} samples")
print(f"Тестовые данные: {x_test.shape[0]} samples")
print(f"Соотношение классов в целевой переменной: {y.value_counts(normalize=True)}")

Тренировочные данные: 2400 samples
Тестовые данные: 600 samples
Соотношение классов в целевой переменной: Target
0    0.806667
1    0.193333
Name: proportion, dtype: float64


In [69]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

# Обучение модели
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Предсказания
y_pred = rf_model.predict(X_test)

# Оценка модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8067

Classification Report:
              precision    recall  f1-score   support

           0       0.81      1.00      0.89       484
           1       0.00      0.00      0.00       116

    accuracy                           0.81       600
   macro avg       0.40      0.50      0.45       600
weighted avg       0.65      0.81      0.72       600



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Итог: Для оценки влияния образа жизни и диеты на психичесткое здоровье требуется значительно больше данных и дополнительных признаков.